# Scraping Halaman Web Google Scholar Menggunakan Library BeautifulSoup

## Kendala:
Solved:
1. Melakukan manipulasi string untuk pembersihan string karena pattern string yang berbeda-beda
2. Membuat multipage dengan melihat pola URL beserta parameter dari URL
3. Output kerap kali tidak muncul ketika sering di-running
4. Memahami struktur HTML dari halaman web Google Scholar secara keseluruhan
5. Mengisi kekosongan value dari list yang diakibatkan elemen HTML yang informasinya tidak sesuai pola
6. Terdapat cara penyajian informasi yang berbeda di setiap web. Misalnya Web Amazon yang sudah menyajikan informasi dari setiap kontainer yang ditampilkan di halaman web dalam bentuk JSON sehingga scraping web dapat dilakukan dengan langsung mengambil data file JSONnya. Sedangkan web Google Scholar ini tidak menerapkan hal serupa sehingga kita harus mengambil datanya melalui struktur HTML langsung.  
<br><br>

Unsolved:<br>
* Tidak ada struktur informasi yang baku pada halaman web untuk dapat dilakukan scraping, salah satunya untuk halaman web Google Scholar. 
* Misalnya dalam kasus ini nama jurnal, tahun, dan author dipisahkan oleh tanda baca (-, :) sehingga sulit untuk mendapatkan masing-masing atribut tersebut secara satuan. Data yang menyatu tersebut kerap kali mengharuskan kita untuk melakukan praproses, salah satunya melalui penggunaan fungsi regex.
* Sedangkan, jika masing-masing atribut tersebut tersedia di tag HTML yang berbeda dan spesifik, maka proses untuk mendapatkannya menjadi lebih mudah.


### Saran
1. Sebaiknya pelajari **REGEX** untuk melakukan manipulasi string
2. Sebaiknya pelajari struktur HTML dari halaman web yang akan discraping dan library-library yang mendukung web scraping
3. Terdapat beberapa data dari web Google Scholar yang sebaiknya perlu diambil karena saat ini belum kami manfaatkan, seperti jumlah sitasi, dll.
4. Sekiranya menemukan cara yang lebih efisien dalam proses scraping, diperkenankan untuk memperbaiki proses scraping yang telah kami buat dan meneruskan ke proses selanjutnya.

### Deklarasi Library dan List yang Digunakan

In [1]:
# mengimport library
from bs4 import BeautifulSoup  # library utama untuk melakukan scrapping
import requests                # library untuk mendapatkan HTTP request
import unicodedata             # library untuk membersihkan unicode yang terbawa
import pandas as pd            # library untuk membuat data frame

from time import sleep         # library untuk melakukan sleep (memberi jeda sekian waktu)
from random import randint     # library untuk melakukan randomisasi integer

import re                      # library untuk menerapkan regular expression (regex)
import statistics              # library untuk menggunakan fungsi statistik

# list
article_title = [] # berisi judul artikel
author_full = []   # berisi author
journal_info = []  # berisi nama jurnal dan tahun (sebelum dipisahkan)
journal_name = []  # berisi nama jurnal
year = []          # berisi tahun
link = []          # berisi link artikel
link_author = []

### Proses Mendapatkan dan Membagi Elemen HTML

In [2]:
# menentukan nilai start (yang ada di parameter) dari setiap halaman web
pages = range(0, 110, 10)
# counter jumlah halaman
count = 1

# untuk setiap nilai start
for page in pages:
    # mengeprint halaman sekarang dan nilai start 
    print("Halaman: " + str(count) + " || Start: " + str(page))
    # menambahkan nilai count
    count = count + 1
    
    # mendapatkan HTTP request untuk halaman web yang dituju
    page = requests.get("https://scholar.google.com/scholar?start="+ str(page) + "&q=information+technology+indonesia+edu+journal+\"ac+id\"+-abstrak&hl=en&as_sdt=0,5")
    # menggunakan library BeautifulSoup untuk mendapatkan data HTML berdasarkan hasil HTTP request
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # memberi jeda terhadap proses iterasi selanjutnya
    sleep(randint(2, 5)) # antara 2 detik hingga 5 detik
    
    # mengambil seluruh kontainer setiap artikel
    gs_div = soup.find_all('div', class_='gs_r gs_or gs_scl')
    # untuk setiap kontainer:
    for container in gs_div:
        # mengambil elemen h3 dengan class gs_rt yang berisi nama artikel
        article_title.append(container.find('h3', class_='gs_rt').text)
        
        # mengambil elemen div dengan class gs_or_ggsm yang berisi link artikel
        gs_or_ggsm = container.find_all("div", class_= "gs_or_ggsm")
        # jika ditemukan kontainer yang memiliki elemen div dengan kelas gs_or_ggsm:
        # ctt: perlu ditemukan elemen div tersebut karena tidak semua artikel memiliki link yang langsung tertuju ke halaman web jurnal
        if(container.find('div', class_='gs_or_ggsm')):
            # dapatkan href dari elemen a
            a = container.find('a', href=True).get('href')
            # tambahkan href dari elemen a ke dalam list 'link'
            link.append(a)
        # jika tidak ditemukan
        else:
            # isikan bagian yang tidak ada tersebut dengan value "None"
            link.append('None')
        
        # mengambil elemen div dengan class gs_a yang berisi nama author, nama jurnal, tahun
        gs_a = container.find('div', class_='gs_a').text
        #print(gs_a)
        gs_a_author = container.find('div', class_='gs_a')
        if(gs_a_author.find('a', href=True)):
            # dapatkan href dari elemen a
            a = gs_a_author.find('a', href=True).get('href')
            # tambahkan href dari elemen a ke dalam list 'link'
            print("https://scholar.google.com" + a)
            link_author.append("https://scholar.google.com" + a)
        else:
            link_author.append('None')
            print('none')
            
        # ambil nama-nama author pakai split (ctt: data yang diperoleh masih kotor)
        a_split = gs_a.split("-")
        
        # normalize() -> untuk bersihin hasil split sebelumnya yang kotor
        # ambil nama author secara satuan pakai split
        clean_author = unicodedata.normalize("NFKD",a_split[0]).split(", ")
        
        # menentukan journal_info yang terdiri dari nama jurnal dan tahun
        if(len(a_split) <= 1):
            journal_info.append("")
        else:
            journal_info.append(a_split[1])
        
        # list untuk menampung nama-nama author dari tiap kontainer
        author_list = []
        # setiap nama author dari nama-nama author:
        for auth in clean_author: 
            # masukkan ke list author
            # rstrip() -> untuk menghilangkan space yg berlebih di sebelah kanan string
            author_list.append(auth.rstrip())
        # menggabungkan nama-nama author yang berada dalam satu kontainer yang sama 
        x = ", ".join(author_list)
        # memasukkan nama-nama author ke dalam list author_full
        author_full.append(x)
    print(end="\n")

Halaman: 1 || Start: 0
https://scholar.google.com/citations?user=9SD1jeoAAAAJ&hl=en&oe=ASCII&oi=sra
https://scholar.google.com/citations?user=MCE67SoAAAAJ&hl=en&oe=ASCII&oi=sra
https://scholar.google.com/citations?user=cqlL0UwAAAAJ&hl=en&oe=ASCII&oi=sra
https://scholar.google.com/citations?user=9SD1jeoAAAAJ&hl=en&oe=ASCII&oi=sra
https://scholar.google.com/citations?user=l_YozBoAAAAJ&hl=en&oe=ASCII&oi=sra
https://scholar.google.com/citations?user=dqb9lzEAAAAJ&hl=en&oe=ASCII&oi=sra
https://scholar.google.com/citations?user=9SD1jeoAAAAJ&hl=en&oe=ASCII&oi=sra
https://scholar.google.com/citations?user=SrabAlgAAAAJ&hl=en&oe=ASCII&oi=sra
https://scholar.google.com/citations?user=a6nTVKgAAAAJ&hl=en&oe=ASCII&oi=sra
https://scholar.google.com/citations?user=_MegyMEAAAAJ&hl=en&oe=ASCII&oi=sra

Halaman: 2 || Start: 10
https://scholar.google.com/citations?user=pqjC3mwAAAAJ&hl=en&oe=ASCII&oi=sra
https://scholar.google.com/citations?user=l_YozBoAAAAJ&hl=en&oe=ASCII&oi=sra
https://scholar.google.com/c

In [3]:
print(link_author)

['https://scholar.google.com/citations?user=9SD1jeoAAAAJ&hl=en&oe=ASCII&oi=sra', 'https://scholar.google.com/citations?user=MCE67SoAAAAJ&hl=en&oe=ASCII&oi=sra', 'https://scholar.google.com/citations?user=cqlL0UwAAAAJ&hl=en&oe=ASCII&oi=sra', 'https://scholar.google.com/citations?user=9SD1jeoAAAAJ&hl=en&oe=ASCII&oi=sra', 'https://scholar.google.com/citations?user=l_YozBoAAAAJ&hl=en&oe=ASCII&oi=sra', 'https://scholar.google.com/citations?user=dqb9lzEAAAAJ&hl=en&oe=ASCII&oi=sra', 'https://scholar.google.com/citations?user=9SD1jeoAAAAJ&hl=en&oe=ASCII&oi=sra', 'https://scholar.google.com/citations?user=SrabAlgAAAAJ&hl=en&oe=ASCII&oi=sra', 'https://scholar.google.com/citations?user=a6nTVKgAAAAJ&hl=en&oe=ASCII&oi=sra', 'https://scholar.google.com/citations?user=_MegyMEAAAAJ&hl=en&oe=ASCII&oi=sra', 'https://scholar.google.com/citations?user=pqjC3mwAAAAJ&hl=en&oe=ASCII&oi=sra', 'https://scholar.google.com/citations?user=l_YozBoAAAAJ&hl=en&oe=ASCII&oi=sra', 'https://scholar.google.com/citations?u

### Mereplace Pattern yang Ditemukan dengan String Kosong Menggunakan Replacement dengan Regex

In [4]:
# perulangan sebanyak panjang list judul artikel
for i in range(len(article_title)):
    # menghapus pattern kurung siku seperti: [PDF], [PERNYATAAN], [HTML]
    t = re.sub(r'\[[^\]]*\]', '', article_title[i])
    # menghapus pattern spasi yang berlebih di awal string judul artikel
    t = re.sub(r"^\s+", '', t)
    # memasukkan string yang sudah bersih ke list article_title
    article_title[i] = t

In [5]:
# perulangan sebanyak panjang list journal_info
for i in range(len(journal_info)):
    # mendapatkan data tahun dari list journal_info dengan menghapus elemen selain numeric
    t = re.sub("[^0-9]", "", journal_info[i])
    # mendapatkan data nama jurnal dari list journal_info dengan menghapus elemen selain huruf dan spasi
    t2 = re.sub("[^a-z|A-Z| ]", "", journal_info[i])
    # menghapus spasi yang berlebih di awal dan akhir nama jurnal
    t2 = "".join(t2.rstrip().lstrip())
    # memasukkan nama tahun dan nama jurmal ke dalam list tahun maupun journal_name
    year.append(t)
    journal_name.append(t2)

In [16]:
len(journal_name)

110

### Mengisi Data yang Kosong

In [6]:
# perulangan sebanyak panjang list tahun
for i in range (len(year)):
    # jika elemen list tahun ada yang tidak berisi
    if(year[i] == ""):
        # isikan nilai kosong tersebut dengan modus dari tahun
        year[i] = statistics.mode(year)

In [7]:
# perulangan sebanyak panjang list journal_name
for i in range (len(journal_name)):
    # jika elemen list journal_name ada yang tidak berisi
    if(journal_name[i] == ""):
        # isikan nilai kosong tersebut dengan string "None"
        journal_name[i] = 'None'

### Membuat Data Frame

In [8]:
df = pd.DataFrame({
    'author' : author_full,
    'year' : year,
    'article_title' : article_title,
    'journal_name' : journal_name,
    'link' : link,
    'link_author' : link_author
})

In [9]:
# Menampilkan 20 data teratas dari data frame
df.tail(20)

,author,year,article_title,journal_name,link,link_author
90,"RTS Hariyati, E Nurachmah...",2011,Developing distance learning method for health...,International Journal of,https://academicjournals.org/journal/IJEAPS/ar...,https://scholar.google.com/citations?user=oB2-...
91,"RF Sari, A Kurniawan",2011,Evaluation of the Implementation of Indonesian...,Asian Journal of Information Technology,https://www.researchgate.net/profile/Riri-Sari...,https://scholar.google.com/citations?user=ZKRr...
92,AQ Pudjiastuti,2020,Cek Similarity Role of Agriculture Sector on t...,repositoryunitriacid,None,None
93,"E Triandini, A Djunaidy, D Siahaan",2008,E-COMMERCE ADOPTION BY SMEs IN INDONESIA: A LI...,International Journal of,https://www.researchgate.net/profile/Evi-Trian...,https://scholar.google.com/citations?user=hIwT...
94,"UA Rachmawati, SC Haryanti, N Aini...",2020,E-health grid network topology based on provin...,and Bio,https://gvpress.com/journals/IJBSBT/vol8_no2/2...,https://scholar.google.com/citations?user=Bf8L...
95,"M Febriana, J Nurkamto...",2018,RETRACTED: Teaching in Rural Indonesian School...,Journal of,None,https://scholar.google.com/citations?user=5ZM3...
96,"R Rahmawati, K Yahiji, C Mahfud...",2018,Chinese ways of being good Muslim: from the Ch...,Indonesian Journal of,http://digilib.uinsby.ac.id/30855/3/Jauharoti%...,https://scholar.google.com/citations?user=5IQ8...
97,"R Syahputra, I Robandi, M Ashari",2015,PSO based multi-objective optimization for rec...,International Journal of,http://repository.umy.ac.id/bitstream/handle/1...,https://scholar.google.com/citations?user=Tn5c...
98,"S Fauzan, SDW Prajanti...",2019,The Effect of Budgeting Quality and Human Reso...,Journal of Economic,https://journal.unnes.ac.id/sju/index.php/jeec...,https://scholar.google.com/citations?user=uZBE...
99,"G Fathoni, SA Widayat, PA Topan, A Jalil...",2017,Comparison of State-of-Charge (SOC) estimation...,Technology,None,https://scholar.google.com/citations?user=GPQY...


In [10]:
df.article_title = df.article_title.str.lower()

In [11]:
df

,author,year,article_title,journal_name,link,link_author
0,"IPG Sukerta, GS Darma",2014,application of information technology integrat...,Jurnal Manajemen Bisnis,http://journal.undiknas.ac.id/index.php/magist...,https://scholar.google.com/citations?user=9SD1...
1,"I Muda, DY Wardani, A Maksum, AF Lubis...",2017,the influence of human resources competency an...,Technology,https://www.researchgate.net/profile/Iskandar-...,https://scholar.google.com/citations?user=MCE6...
2,"AM Erzad, S Suciati",2018,the existence of kudus islamic local culture t...,QIJIS Qudus International Journal of,None,https://scholar.google.com/citations?user=cqlL...
3,"IGPA Artawijaya, GS Darma",2015,evaluation the effectivity of performance mana...,Jurnal Manajemen Bisnis,http://journal.undiknas.ac.id/index.php/magist...,https://scholar.google.com/citations?user=9SD1...
4,"D Soyusiawaty, Y Zakaria",2018122018,book data content similarity detector with cos...,th International Conference,None,https://scholar.google.com/citations?user=l_Yo...
...,...,...,...,...,...,...
105,"T Winarso, HS Disemadi...",2020,protection of private data consumers p2p lendi...,Tadulako Law,http://jurnal.untad.ac.id/jurnal/index.php/TLR...,https://scholar.google.com/citations?user=qJUi...
106,LL aaS SSSSSS,2009,m. kamalageo. ugm. ac. id department of geogra...,Indonesian Journal of Geography,https://core.ac.uk/download/pdf/298729827.pdf,None
107,CT Anni,2018,school counselors' intention to use technology...,Turkish Online Journal of Educational Technology,https://files.eric.ed.gov/fulltext/EJ1176168.pdf,https://scholar.google.com/citations?user=00pi...
108,M Mustika,2021,hasil turnitin_examining the impact of corpora...,None,None,None


In [12]:
df[df['article_title'].str.contains("data")]

,author,year,article_title,journal_name,link,link_author
4,"D Soyusiawaty, Y Zakaria",2018122018,book data content similarity detector with cos...,th International Conference,None,https://scholar.google.com/citations?user=l_Yo...
11,"D Soyusiawaty, Y Zakaria",2018122018,book data content similarity detector with cos...,th International Conference,None,https://scholar.google.com/citations?user=l_Yo...
40,A Ahmadi,2019,the use of sinta (science and technology index...,Journal of Mechanical Engineering and Technology,https://www.academia.edu/download/60405706/IJM...,https://scholar.google.com/citations?user=pejs...
78,"I Busnetty, TTH Tambunan",2020,gap between regions in the use of e-commerce b...,Journal of Telecommunications and,None,https://scholar.google.com/citations?user=Meu2...
80,KM Hindrayani,2021,determining students preparation for college e...,IJCONSIST JOURNALS,https://www.ijconsist.org/index.php/ijconsist/...,https://scholar.google.com/citations?user=mGis...
87,"A Yanuar, A Mun'im, ABA Lagho, RR Syahdi...",2011,medicinal plants database and three dimensiona...,arXiv preprint arXiv,https://arxiv.org/pdf/1111.7183,https://scholar.google.com/citations?user=y3Rn...
101,"F Rahman, DA Wicaksono",2021,researching references on interpretation of pe...,Hukum De Jure,None,https://scholar.google.com/citations?user=eKYJ...
105,"T Winarso, HS Disemadi...",2020,protection of private data consumers p2p lendi...,Tadulako Law,http://jurnal.untad.ac.id/jurnal/index.php/TLR...,https://scholar.google.com/citations?user=qJUi...


In [13]:
df[df['article_title'].str.contains("web")]

,author,year,article_title,journal_name,link,link_author
6,"CR Dewi, GS Darma",2014,"website usability, satisfaction, loyalty, secu...",Jurnal Manajemen Bisnis,http://journal.undiknas.ac.id/index.php/magist...,https://scholar.google.com/citations?user=9SD1...
32,JS Pasaribu,2021,development of a web based inventory informati...,of Engineering Science and Information Technology,http://ijesty.org/index.php/ijesty/article/dow...,https://scholar.google.com/citations?user=I1LN...
46,"M Anshori, I Maurits...",2014,effect of social media on website city governm...,Gunadarma Ac Id,http://achsan.staff.gunadarma.ac.id/Publicatio...,https://scholar.google.com/citations?user=2dO2...
49,"GF Nama, M Ulvan, A Ulvan...",2015,design and implementation web based geographic...,Information Technology,https://www.researchgate.net/profile/Gigih-For...,https://scholar.google.com/citations?user=3pCL...


In [ ]:
df[df['article_title'].str.contains("multimedia")]

In [ ]:
df[df['article_title'].str.contains("cyber")]

In [ ]:
df[df['article_title'].str.contains("cloud")]

In [ ]:
df[df['article_title'].str.contains("information system")]

In [ ]:
len(df)